In [198]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [199]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [200]:
train_data['Group'] = train_data['PassengerId'].str[:4].astype(float)
train_data['PeopleId'] = train_data['PassengerId'].str[-2:].astype(float)

test_data['Group'] = test_data['PassengerId'].str[:4].astype(float)
test_data['PeopleId'] = test_data['PassengerId'].str[-2:].astype(float)

train_data = train_data.drop('PassengerId', axis=1)
test_data = test_data.drop('PassengerId', axis=1)

In [201]:
train_data['Data'] = "train"
test_data['Data'] = "test"

all_data = pd.concat([train_data,test_data], axis=0)

all_data.replace({False: 0, True: 1}, inplace=True)

In [202]:
all_data.head(15)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,PeopleId,Data
0,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0,1.0,1.0,train
1,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0,2.0,1.0,train
2,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0,3.0,1.0,train
3,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0,3.0,2.0,train
4,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0,4.0,1.0,train
5,Earth,0.0,F/0/P,PSO J318.5-22,44.0,0.0,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,1.0,5.0,1.0,train
6,Earth,0.0,F/2/S,TRAPPIST-1e,26.0,0.0,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,1.0,6.0,1.0,train
7,Earth,1.0,G/0/S,TRAPPIST-1e,28.0,0.0,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,1.0,6.0,2.0,train
8,Earth,0.0,F/3/S,TRAPPIST-1e,35.0,0.0,0.0,785.0,17.0,216.0,0.0,Andona Beston,1.0,7.0,1.0,train
9,Europa,1.0,B/1/P,55 Cancri e,14.0,0.0,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,1.0,8.0,1.0,train


In [203]:
from sklearn.preprocessing import MinMaxScaler
scale_columns = ['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
scaler = MinMaxScaler()
all_data[scale_columns] = pd.DataFrame(scaler.fit_transform(all_data[scale_columns]), columns=scale_columns)

In [204]:
test = all_data[['Age','HomePlanet','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].copy()

test['Age'].fillna(all_data['Age'].mode(), inplace=True)
test['HomePlanet'].fillna(all_data['HomePlanet'].mode()[0], inplace=True)
test['VIP'].fillna(all_data['VIP'].mode()[0], inplace=True)

df = pd.get_dummies(test['HomePlanet'])
test = pd.concat([test, df], axis=1)

In [205]:
fillcolumns = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
for column in fillcolumns:
    columnlist = ['Age','HomePlanet','VIP']
    columnlist.append(column)
    train = all_data[columnlist].copy()
    train.dropna(inplace=True)

    df = pd.get_dummies(train['HomePlanet'])
    train = pd.concat([train,df], axis=1)

    x_test = test[['Age','VIP','Earth','Europa','Mars']]

    y_data = train[column]
    x_data = train.drop([column,'HomePlanet'], axis=1)

    from sklearn.model_selection import train_test_split
    x_train, x_val, y_train, y_val = train_test_split(x_data,y_data,test_size=0.1,random_state=42)

    from xgboost import XGBRegressor

    xgb = XGBRegressor(n_estimators=500, early_stopping_rounds=5)
    xgb.fit(x_train,y_train,eval_set=[(x_val,y_val)])

    result = xgb.predict(x_test)

    x_test[column] = result
    all_data[column].fillna(x_test[column], inplace=True)

[0]	validation_0-rmse:0.34153
[1]	validation_0-rmse:0.24103
[2]	validation_0-rmse:0.17154
[3]	validation_0-rmse:0.12402
[4]	validation_0-rmse:0.09224
[5]	validation_0-rmse:0.07174
[6]	validation_0-rmse:0.05926
[7]	validation_0-rmse:0.05205
[8]	validation_0-rmse:0.04818
[9]	validation_0-rmse:0.04621
[10]	validation_0-rmse:0.04517
[11]	validation_0-rmse:0.04470
[12]	validation_0-rmse:0.04447
[13]	validation_0-rmse:0.04438
[14]	validation_0-rmse:0.04435
[15]	validation_0-rmse:0.04435
[16]	validation_0-rmse:0.04436
[17]	validation_0-rmse:0.04434
[18]	validation_0-rmse:0.04433
[19]	validation_0-rmse:0.04435
[20]	validation_0-rmse:0.04437
[21]	validation_0-rmse:0.04437
[22]	validation_0-rmse:0.04438
[0]	validation_0-rmse:0.34310
[1]	validation_0-rmse:0.24308
[2]	validation_0-rmse:0.17409
[3]	validation_0-rmse:0.12720
[4]	validation_0-rmse:0.09613
[5]	validation_0-rmse:0.07624
[6]	validation_0-rmse:0.06440
[7]	validation_0-rmse:0.05754
[8]	validation_0-rmse:0.05399
[9]	validation_0-rmse:0.052

C:\Users\anike\AppData\Local\Temp\ipykernel_21712\3928520015.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = result
C:\Users\anike\AppData\Local\Temp\ipykernel_21712\3928520015.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = result


[0]	validation_0-rmse:0.34414
[1]	validation_0-rmse:0.24112
[2]	validation_0-rmse:0.16932
[3]	validation_0-rmse:0.11953
[4]	validation_0-rmse:0.08530
[5]	validation_0-rmse:0.06217
[6]	validation_0-rmse:0.04703
[7]	validation_0-rmse:0.03762
[8]	validation_0-rmse:0.03214
[9]	validation_0-rmse:0.02919
[10]	validation_0-rmse:0.02770
[11]	validation_0-rmse:0.02701
[12]	validation_0-rmse:0.02672
[13]	validation_0-rmse:0.02659
[14]	validation_0-rmse:0.02655
[15]	validation_0-rmse:0.02655
[16]	validation_0-rmse:0.02655
[17]	validation_0-rmse:0.02656
[18]	validation_0-rmse:0.02658
[19]	validation_0-rmse:0.02659
[0]	validation_0-rmse:0.34405
[1]	validation_0-rmse:0.24285
[2]	validation_0-rmse:0.17278
[3]	validation_0-rmse:0.12474
[4]	validation_0-rmse:0.09242
[5]	validation_0-rmse:0.07130
[6]	validation_0-rmse:0.05830
[7]	validation_0-rmse:0.05058
[8]	validation_0-rmse:0.04629
[9]	validation_0-rmse:0.04412
[10]	validation_0-rmse:0.04309
[11]	validation_0-rmse:0.04260
[12]	validation_0-rmse:0.042

C:\Users\anike\AppData\Local\Temp\ipykernel_21712\3928520015.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = result
C:\Users\anike\AppData\Local\Temp\ipykernel_21712\3928520015.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = result


[0]	validation_0-rmse:0.34390
[1]	validation_0-rmse:0.24269
[2]	validation_0-rmse:0.17253
[3]	validation_0-rmse:0.12447
[4]	validation_0-rmse:0.09214
[5]	validation_0-rmse:0.07101
[6]	validation_0-rmse:0.05798
[7]	validation_0-rmse:0.05039
[8]	validation_0-rmse:0.04622
[9]	validation_0-rmse:0.04402
[10]	validation_0-rmse:0.04287
[11]	validation_0-rmse:0.04233
[12]	validation_0-rmse:0.04206
[13]	validation_0-rmse:0.04188
[14]	validation_0-rmse:0.04181
[15]	validation_0-rmse:0.04176
[16]	validation_0-rmse:0.04179
[17]	validation_0-rmse:0.04177
[18]	validation_0-rmse:0.04175
[19]	validation_0-rmse:0.04180
[20]	validation_0-rmse:0.04185
[21]	validation_0-rmse:0.04185
[22]	validation_0-rmse:0.04184


C:\Users\anike\AppData\Local\Temp\ipykernel_21712\3928520015.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[column] = result


In [206]:
def fillna_column(dataset,column):
    dataset_clean = dataset[dataset[column].notna()]
    dict = pd.Series(dataset_clean[column].values,index=dataset_clean.Group).to_dict()
    dataset['New'] = dataset['Group'].map(dict)
    dataset[column].fillna(dataset['New'], inplace=True)
    dataset = dataset.drop('New', axis=1)
    return dataset

all_data = fillna_column(all_data,'Cabin')
all_data = fillna_column(all_data,'HomePlanet')

all_data['Cabin'].fillna(method="ffill", inplace=True)
all_data['HomePlanet'].fillna(method="ffill", inplace=True)

In [207]:
for col in ['CryoSleep','Destination','Age','VIP']:
    all_data[col].fillna(all_data[col].mode()[0], inplace=True)

In [208]:
all_data['Deck'] = all_data['Cabin'].str[0]
all_data['Num'] = all_data['Cabin'].str[2:-2].astype(float)
all_data['Side'] = all_data['Cabin'].str[-1]

all_data = all_data.drop(['Name','Cabin'], axis=1)

In [209]:
categorical_columns = ['HomePlanet','Destination','Deck','Side']

all_data = pd.get_dummies(all_data, columns=categorical_columns)

In [210]:
train_data = all_data[all_data['Data']=="train"]
test_data = all_data[all_data['Data']=="test"]

train_data = train_data.drop('Data', axis=1)
test_data = test_data.drop(['Data','Transported'], axis=1)

In [211]:
test_data.isna().sum()

CryoSleep                    0
Age                          0
VIP                          0
RoomService                  0
FoodCourt                    0
ShoppingMall                 0
Spa                          0
VRDeck                       0
Group                        0
PeopleId                     0
Num                          0
HomePlanet_Earth             0
HomePlanet_Europa            0
HomePlanet_Mars              0
Destination_55 Cancri e      0
Destination_PSO J318.5-22    0
Destination_TRAPPIST-1e      0
Deck_A                       0
Deck_B                       0
Deck_C                       0
Deck_D                       0
Deck_E                       0
Deck_F                       0
Deck_G                       0
Deck_T                       0
Side_P                       0
Side_S                       0
dtype: int64

In [212]:
train_data.to_csv('cleaned_train.csv', index=False)
test_data.to_csv('cleaned_test.csv', index=False)